# Análisis Exploratorio de Datos (EDA)

Mediante la exploración detallada deL conjunto de datos proporcionado por el Observatorio de Movilidad y Seguridad Vial (OMSV), se busca identificar patrones, tendencias y factores clave que contribuyen a la ocurrencia de siniestros y a la gravedad de sus consecuencias. Este análisis preliminar no solo sienta las bases para el diseño del dashboard y la definición de KPIs, sino que también ofrece perspectivas críticas que respaldarán la formulación de políticas efectivas y medidas preventivas para reducir el número de víctimas fatales en siniestros viales en la ciudad.

### Se importan las librerías necesarias para el proceso de EDA

In [61]:
# Importar librerías
import pandas as pd # Importa la librería pandas para el manejo de datos
import numpy as np # Importa la librería numpy para el manejo de datos
import calendar # Importa la librería calendar para trabajar con fechas
import locale # Importa la librería locale para configurar la configuración regional
import seaborn as sns # Importa la librería seaborn para la visualización de datos
import matplotlib.pyplot as plt # Importa la librería matplotlib.pyplot para la generación de gráficos
from datetime import datetime # Importa la clase datetime del módulo datetime para trabajar con fechas y horas
import warnings # Importa la función filterwarnings del módulo warnings para ignorar las advertencias
warnings.filterwarnings("ignore")
import utils # Script con funciones de ETL y EDA

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Extracción de los datos desde un archivo xlsx, que contiene varias hojas (sheets), las de interés para el proyecto son HECHOS y VICTIMAS respectivamente. Se hará uso de algunas funciones de ETL y EDA recopiladas en el Script `utils.py`

In [62]:
hechos = pd.read_excel("data\homicidios.xlsx", sheet_name="HECHOS")
victimas = pd.read_excel("data\homicidios.xlsx", sheet_name="VICTIMAS")

### Exploración Dataframe `hechos`

In [63]:
hechos.head()  # Visualizamos los primeros 5 registros

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [64]:
hechos.shape  # Tamaño del dataset

(696, 21)

In [65]:
# Eliminamos vacíos dado que exista un registro con todos sus datos vacíos
hechos = hechos.dropna(how="all")

# Eliminamos registros duplicados
hechos = hechos.drop_duplicates()  # Eliminamos duplicados

# Revisamos el tamaño del dataframe
hechos.shape

(696, 21)

Observaciones: 

* Se Observa que no existen columnas y filas con todos sus valores vacíos o duplicados. 

### Verificamos el tipo de datos, y el total de nulos por columna en porcentaje.

In [66]:
utils.verificar_tipo_datos_y_nulos(hechos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
11,Altura,[<class 'float'>],18.53,81.47,567
12,Cruce,"[<class 'str'>, <class 'float'>]",75.43,24.57,171
13,Dirección Normalizada,"[<class 'str'>, <class 'float'>]",98.85,1.15,8
10,Calle,"[<class 'str'>, <class 'float'>]",99.86,0.14,1
19,VICTIMA,[<class 'str'>],100.00,0.00,0
18,PARTICIPANTES,[<class 'str'>],100.00,0.00,0
17,pos y,[<class 'str'>],100.00,0.00,0
16,pos x,[<class 'str'>],100.00,0.00,0
15,XY (CABA),[<class 'str'>],100.00,0.00,0
14,COMUNA,[<class 'int'>],100.00,0.00,0


Observaciones: 

* Es necesario renombrar columnas para manejar un solo formato de nombres de campos.
* Se elimina el campo ALTURA por tener una gran cantidad de nulos y no poder imputar.
* El campo cruce hace referencia a si es cruce o no, se imputan los valores vacíos con "No" y si efectivamente es un cruce con "Si".
* La columna "Dirección normalizada" se imputará con ayuda de la columna "lugar del hecho". 
* Las columnas 'HORA', 'HH' y 'Calle' presentan varios tipos de datos en la misma columna, por lo que se deben revisar cada uno y asignar un único tipo de dato.

### Renombrar columnas

In [67]:
# Renombramos columnas
hechos.rename(
    columns={
        "ID": "id",
        "N_VICTIMAS": "num_victimas",
        "FECHA": "fecha",
        "AAAA": "anio",
        "MM": "mes",
        "DD": "dia",
        "HORA": "hora",
        "HH": "franja_horaria",
        "LUGAR_DEL_HECHO": "lugar",
        "TIPO_DE_CALLE": "tipo_calle",
        "COMUNA": "comuna",
        "XY (CABA)": "punto_coordenado",
        "pos x": "longitud",
        "pos y": "latitud",
        "PARTICIPANTES": "participantes",
        "VICTIMA": "victima",
        "ACUSADO": "acusado",
        "Dirección Normalizada": "direccion_normalizada",
        "Calle": "calle",
        "Altura": "altura",
        "Cruce": "cruce",
    },
    inplace=True,
)

In [68]:
# Observamos las columnas renombradas
hechos.columns

Index(['id', 'num_victimas', 'fecha', 'anio', 'mes', 'dia', 'hora',
       'franja_horaria', 'lugar', 'tipo_calle', 'calle', 'altura', 'cruce',
       'direccion_normalizada', 'comuna', 'punto_coordenado', 'longitud',
       'latitud', 'participantes', 'victima', 'acusado'],
      dtype='object')

### Eliminación de columna `altura`, debido al alto número de valores nulos

In [69]:
# Eliminar la columna ALTURA
hechos = hechos.drop(["altura"], axis=1)

### Imputación de datos nulos en la columna `cruce`, los vacíos se cambian por "No" y los demás por "Si"

In [70]:
# Se reemplazan los valores no nulos por "no" y los nulos por "si"
hechos["cruce"] = np.where(hechos["cruce"].notnull(), "Si", "No")
hechos["cruce"]

0      Si
1      Si
2      No
3      Si
4      Si
       ..
691    Si
692    Si
693    Si
694    No
695    No
Name: cruce, Length: 696, dtype: object

### Imputación de datos nulos en la columna `direccion_normalizada`, los vacíos se cambian por "Sin dato", debido a que no hay información que permita completar de forma acertada estos datos faltantes, aunque solo representan un 1% del dataset, no se considera eliminar sus registros para preservar la mayor información posible

In [71]:
# Se rellenan nulos con SD
hechos["direccion_normalizada"].fillna("SD", inplace=True)

# Se verifican los datos imputados
hechos[hechos["direccion_normalizada"] == "SD"]["direccion_normalizada"]

38     SD
106    SD
119    SD
180    SD
181    SD
313    SD
546    SD
621    SD
Name: direccion_normalizada, dtype: object

### Tipos de variables (hora, franja_horaria, calle)

### hora

In [72]:
# Cantidad de valores por tipo de dato en la columna 'hora'
hechos["hora"].apply(type).value_counts()

hora
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

Conclusión
- Dado que la mayoría de los datos son del tipo datetime.time, se decide convertir todos a este tipo de dato.

In [73]:
# Se cambia el tipo de dato
hechos["hora"] = hechos["hora"].apply(
    lambda x: utils.convertir_a_time(x)
)  # Se usa la función convertir_a_time de Utils

# Se verifica la cantidad de valores por tipo de dato en la columna 'hora'
hechos["hora"].apply(type).value_counts()

hora
<class 'datetime.time'>    695
<class 'NoneType'>           1
Name: count, dtype: int64

Se observa cuál es el registro que presenta NoneType 

In [74]:
hechos[hechos["hora"].isna()]

,id,num_victimas,fecha,anio,mes,dia,hora,franja_horaria,lugar,tipo_calle,calle,cruce,direccion_normalizada,comuna,punto_coordenado,longitud,latitud,participantes,victima,acusado
518,2019-0103,1,2019-12-18,2019,12,18,None,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",Si,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


Se imputa el valor de faltante con las horas más frecuentes del dataset, es decir la Moda

In [75]:
# Calcula la hora mas común
moda_hora = hechos["hora"].mode().iloc[0]

# Se reemplaza el valor None por la hora más común
hechos["hora"].fillna(moda_hora, inplace=True)

# Se verifica el tipo de dato para la columna
hechos["hora"].apply(type).value_counts()

hora
<class 'datetime.time'>    696
Name: count, dtype: int64

Ya todos los datos son del tipo datetime.time

### franja_horaria

In [76]:
# Cantidad de valores por tipo de dato en la columna 'hora'
hechos["franja_horaria"].apply(type).value_counts()

franja_horaria
<class 'int'>    695
<class 'str'>      1
Name: count, dtype: int64

In [77]:
# Observamos el valor que es string
hechos[hechos["franja_horaria"] == "SD"]

,id,num_victimas,fecha,anio,mes,dia,hora,franja_horaria,lugar,tipo_calle,calle,cruce,direccion_normalizada,comuna,punto_coordenado,longitud,latitud,participantes,victima,acusado
518,2019-0103,1,2019-12-18,2019,12,18,09:00:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",Si,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [78]:
# Cambiamos el valor de la franja_horaria de SD a el entero de la columna 'hora'
hechos.loc[518, "franja_horaria"] = 9

# Verificamos el cambio hecho
hechos["franja_horaria"].apply(type).value_counts()

franja_horaria
<class 'int'>    696
Name: count, dtype: int64

Ya todos los datos son del tipo integer

### calle

In [79]:
# Cantidad de valores por tipo de dato en la columna 'hora'
hechos["calle"].apply(type).value_counts()

calle
<class 'str'>      695
<class 'float'>      1
Name: count, dtype: int64

In [80]:
# Observamos el registro que al parecer era float y vemos que corresponde a NaN
valor_float = hechos[hechos["calle"].apply(lambda x: isinstance(x, (int, float)))]
valor_float

,id,num_victimas,fecha,anio,mes,dia,hora,franja_horaria,lugar,tipo_calle,calle,cruce,direccion_normalizada,comuna,punto_coordenado,longitud,latitud,participantes,victima,acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,NaN,No,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD


In [81]:
# Cambiamos el registro vacío de la columna `calle` por el valor "Sin dato"
hechos.loc[119, "calle"] = "SD"

# Verificamos el cambio hecho
hechos["calle"].apply(type).value_counts()

calle
<class 'str'>    696
Name: count, dtype: int64

Ya todos los datos son del tipo string

### Verificamos nuevamente todas la columnas

In [82]:
utils.verificar_tipo_datos_y_nulos(hechos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,id,[<class 'str'>],100.0,0.0,0
1,num_victimas,[<class 'int'>],100.0,0.0,0
18,victima,[<class 'str'>],100.0,0.0,0
17,participantes,[<class 'str'>],100.0,0.0,0
16,latitud,[<class 'str'>],100.0,0.0,0
15,longitud,[<class 'str'>],100.0,0.0,0
14,punto_coordenado,[<class 'str'>],100.0,0.0,0
13,comuna,[<class 'int'>],100.0,0.0,0
12,direccion_normalizada,[<class 'str'>],100.0,0.0,0
11,cruce,[<class 'str'>],100.0,0.0,0


Conclusiones:

* Se normalizó los nombres de las columnas
* Se dejó un solo tipo de data para cada columna
* Se eliminaron o imputaron datos nulos

## Diccionario de datos

| Variables     | Descripción                                                                                      |
|------------------------------|--------------------------------------------------------------------------------------------------|
| id                           | Identificador único del siniestro                                                              |
| num_victimas                   | Cantidad de víctimas                                                                            |
| fecha                        | Fecha en formato dd/mm/aaaa                                                                     |
| anio                         | Año                                                                                              |
| mes                          | Mes                                                                                              |
| dia                           | Día del mes                                                                                      |
| hora                         | Hora del siniestro                                                                              |
| franja_horaria                           | Franja horaria entera                                                                           |
| lugar              | Dirección del hecho                                                                             |
| tipo_calle                | Tipo de arteria. En el caso de intersecciones a nivel se clasifica según la de mayor jerarquía  |
| calle                        | Nombre de la arteria donde se produjo el hecho                                                  |
| Cruce                        | Cruce en caso de que sea una encrucijada                                                        |
| direccion_normalizada        | Dirección en formato normalizado USIG                                                           |
| comuna                       | Comuna de la ciudad (1 a 15)                                                                    |
| punto_coordenado                    | Geocodificación plana                                                                           |
| longitud                      | Longitud con separador punto. WGS84                                                             |
| latitud                        | Latitud con separador punto. WGS84                                                              |
| participantes                | Conjunción de víctima y acusado                                                                 |
| victima                      | Vehículo que ocupaba quien haya fallecido o se haya lastimado a raíz del hecho, o bien peatón/a. Clasificación agregada del tipo de vehículos. |
| acusado                      | Vehículo que ocupaba quien resultó acusado/a del hecho, sin implicar culpabilidad legal        |
                                      


### Revisión categorías `acusado` y `victima`

Teniendo la siguiente información:

| Variables      | Valores      | Descripción                                                                                                                                                                                                                                                                                                                                                                                                          |
|----------------|--------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|                                                                                                   
| VICTIMA        | PEATON       | Víctima distinta de cualquier ocupante de un vehículo, ya sea un conductor/a o un pasajero/a. Se incluyen los ocupantes o personas que empujan o arrastran un coche de bebé o una silla de ruedas o cualquier otro vehículo sin motor de pequeñas dimensiones. Se incluyen también las personas que caminan empujando una bicicleta o un ciclomotor.                                      |
|                | MOTO         | Vehículo a motor no carrozado que incluye motocicleta, ciclomotor y cuatriciclo.                                                                                                                                                                                                                                                                                                                                     |
|                | AUTO         | Vehículo a motor destinado al transporte de personas, diferente de los motovehículos, y que tenga hasta nueve plazas (incluyendo al asiento del conductor) (Sedan, SUV, coupe, etc).                                                                                                                                                                                                                                |
|                | CARGAS       | Vehículo a motor destinado al transporte de cargas, incluye camiones pesados (con o sin acoplado o semirremolque, etc., camión de recolección de residuos) y livianos (utilitarios, furgonetas, pick-ups, camioneta con caja de carga).                                                                                                                                                                              |
|                | BICICLETA    | Vehículo con al menos dos ruedas, que generalmente es accionado por el esfuerzo muscular de las personas que lo ocupan, en particular mediante pedales o manivelas. Incluye bicicletas de pedaleo asistido y/o con motor.                                                                                                                                                                                          |
|                | PASAJEROS    | Personas lesionadas que se encuentran dentro, descendiendo o ascendiendo de las unidades de autotrasporte público de pasajeros/as y ómnibus de larga distancia.                                                                                                                                                                                                                                                    |
|                | MOVIL        | Vehículos de emergencia: móviles policiales, ambulancias, autobombas.                                                                                                                                                                                                                                                                                                                                               |
|                | OTRO         | Otros vehículos.                                                                                                                                                                                                                                                                                                                                      |
|                | SD           | Sin datos sobre el tipo de víctima.                                                                                                                                                                                                                                                                                                                   |
| ACUSADO        | AUTO         | Vehículo a motor destinado al transporte de personas, diferente de los motovehículos, y que tenga hasta nueve plazas (incluyendo al asiento del conductor) (Sedan, SUV, coupe, etc).                                                                                                                                                                                                                                |
|                | BICICLETA    | Vehículo con al menos dos ruedas, que generalmente es accionado por el esfuerzo muscular de las personas que lo ocupan, en particular mediante pedales o manivelas. Incluye bicicletas de pedaleo asistido y/o con motor.                                                                                                                                                                                          |
|                | CARGAS       | Vehículo a motor destinado al transporte de cargas, incluye camiones pesados (con o sin acoplado o semirremolque, etc., camión de recolección de residuos) y livianos (utilitarios, furgonetas, pick-ups, camioneta con caja de carga).                                                                                                                                                                              |
|                | MOTO         | Vehículo a motor no carrozado que incluye motocicleta, ciclomotor y cuatriciclo.                                                                                                                                                                                                                                                                                                                                     |
|                | OBJETO FIJO  | Colisión contra objetos inmóviles fijados de manera permanente o semipermanente (columna, árbol, semáforo, etc.) o pérdidas de equilibrio de vehículos de dos ruedas que desencadenen la caída de sus ocupantes.                                                                                                                                                                                                       |
|                | PASAJEROS    | Personas lesionadas que se encuentran dentro, descendiendo o ascendiendo de las unidades de autotrasporte público de pasajeros/as y ómnibus de larga distancia.                                                                                                                                                                                                                                                    |
|                | TREN         | Equipo móvil que se desplaza exclusivamente sobre rieles.                                                                                                                                                                                                                                                                                              |
|                | OTRO         | Otros vehículos.                                                                                                                                                                                                                                                                                                                                      |
|                | SD           | Sin datos sobre el vehículo participante.                                                                                                                                                                                                                                                                                                             |
| PARTICIPANTES  | MULTIPLE     | Cuando participan más de un vehículo como contraparte de la víctima.                                                                                                                                                                                                                                                                                   |


Es necesario comparar la información de las tablas anteriores (diccionarios de datos), con los registros del dataset `homicidios` para los campos `acusado` y `victima`

In [83]:
hechos["victima"].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

Observación:

* Se observan dos categorías en el dataset que no se encuentran el diccionario de datos para la columna `victima`: 'OBJETO FIJO', 'PEATON_MOTO'

Se observan los registros:

In [84]:
hechos[hechos["victima"].isin(["OBJETO FIJO", "PEATON_MOTO"])]

,id,num_victimas,fecha,anio,mes,dia,hora,franja_horaria,lugar,tipo_calle,calle,cruce,direccion_normalizada,comuna,punto_coordenado,longitud,latitud,participantes,victima,acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",Si,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,Si,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


Se observan dos registros, uno por cada término distinto al del diccionario de datos. Se decide imputar el valor 'OTRO' dado que no hay información suficiente que pueda indicar el vehículo de la victima.

In [85]:
# Se cambia por el valor 'OTRO'
hechos["victima"] = hechos["victima"].replace(
    {"OBJETO FIJO": "OTRO", "PEATON_MOTO": "OTRO"}
)

# Se verifican los registros
hechos[hechos["id"].isin(["2017-0108", "2020-0063"])]

,id,num_victimas,fecha,anio,mes,dia,hora,franja_horaria,lugar,tipo_calle,calle,cruce,direccion_normalizada,comuna,punto_coordenado,longitud,latitud,participantes,victima,acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",Si,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OTRO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,Si,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,OTRO,MOTO


### Variables Geográficas